In [15]:
import time
import requests 
import hmac
import json
import hashlib
import numpy as np
from datetime import datetime
import pandas as pd
import json
import os
from datetime import datetime
import threading
#----------
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
#----------
from IPython.display import clear_output

# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = config['API']['key']
API_SECRET = bytes(config['API']['secret'], 'utf-8') 

symbol = config['GRID_SYSTEM']['symbol']

In [5]:
###############################################################################################
#--------------------------------------  Bitkub  ---------------------------------------------#
###############################################################################################

In [6]:
class bitkubAPI():
    def __init__(self,host,api_key,api_secret):
        self.API_HOST = host
        self.API_KEY = api_key
        self.API_SECRET = api_secret
        self.header = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'X-BTK-APIKEY': self.API_KEY,
            }
        
    def timestamp(self,tm):
        date = datetime.datetime.strptime(tm, "%Y-%m-%d")
        ts = time.mktime(date.timetuple())
        return ts
        
    def _json_encode(self,data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def _sign(self,data):
        j = self._json_encode(data)
        h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()
    
    def _get(self,url,**kwargs): 
        try:
            res = requests.get(self.API_HOST+ url,**kwargs)
            res = json.loads(res.text)
            res = res
            return res
        except:
            return res
    
    def _post(self,url,data): 
        try:
            signature = self._sign(data)
            data['sig'] = signature
            res = requests.post(self.API_HOST + url, headers= self.header, data=self._json_encode(data))
            return res
        except:
            return res
        
    def api_status(self):  
        return self._get('/api/status')
        
    def serverTime(self):
        res = requests.get(self.API_HOST + '/api/servertime')
        return json.loads(res.text)
    
    def bids(self,sym):
        return self._get('/api/market/bids?sym='+sym+'&lmt=1')

    def asks(self,sym):
        return self._get('/api/market/asks?sym='+sym+'&lmt=1')
    
    def balance(self):
        data = {
            'ts': self.getServerTime(),
        }
        return self._post('/api/market/balances',data)
    
    def opened_orders(self,sym):
        data = {
            'sym': sym,
        }
        return self._post('/api/market/balances',data)
    
    def info_orders(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/balances',data)
    
    def cancel_order(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/cancel-order',data)
    
    def his_price(self,sym,frm):
        params = 'sym='+sym
        params = params + '&int=86400'
        params = params + f'&frm={int(self.timestamp(frm))}'
        params = '/api/market/tradingview?'+params
        return self._get(params)
    
    def place_order(self,sym,side,amt,rat,typ):
        data = {
            'sym': sym,
            'amt':amt,
            'rat':rat,
            'typ':typ,
        }
        if(side =='buy'):
            return self._post('/api/market/place-ask/test',data)
        elif(side =='sell'):
            return self._post('/api/market/place-bid/test',data)

In [7]:
###############################################################################################
#-----------------------------------------FTX API---------------------------------------------#
###############################################################################################

In [58]:
import time
import hmac
from typing import Optional, Dict, Any, List
from requests import Request, Session, Response

class ftxAPI():
    def __init__(self):
        self._api_host = 'https://ftx.com/api'
        self._api_key = b'1vyg6pC1WY-ODMhSyeHasYBV8ZVrgpAKYEu4uqWZ'
        self._api_secret  = 'qXuNtAcsQfZZEbzHlrm_rdBOtCrGgFHrYjdkshw8'
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self._session = Session()
        
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, params=params)
    
    def _request(self, method: str, path: str, **kwargs) -> Any:
        request = Request(method, self._api_host + path, **kwargs)
        self._sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _sign_request(self, request: Request) -> None:
        self.ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{self.ts}{prepared.method}{prepared.path_url}'.encode()
        if prepared.body:
            signature_payload += prepared.body
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(self.ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    
    def get_ticker(self,market_name):
        return self._get(f'/markets/{market_name}/orderbook?depth={1}')
    '''
        def place_orders(self,market,side,size,price,type_ord):
            return self._post(f'/orders',{   'market': market,
                                             'side': side,
                                             'price': price,
                                             'size': size,
                                             'type': type_ord,
                                         })
    '''

    
    def place_conditional_orders(self,market,side,size,type_ord,open_price,tp):
        return self._post(f'/conditional_orders',{    'market': market, 
                                                      'side': side,
                                                      'size': size,
                                                      'type': type_ord,
                                                      'triggerPrice':tp,
                                                      'orderPrice':open_price
                                                 })
    
    def historicalPrice(self,market_name,resolution,limit,start_time,end_time):
        return self._get(f'/markets/{market_name}/candles?resolution={resolution}&limit={limit}&start_time={start_time}&end_time={end_time}')
    
    def get_open_orders(self,market):
        return self._get(f'/orders?market={market}')
    
    def place_orders(self,market,side,size,price,type_ord):
        tm = time.localtime() # get struct_time
        time_string = time.strftime("%Y-%m-%d, %H:%M:%S", tm)
        tr = self.get_ticker(market)
        if(side == 'buy'):
            price = tr['asks'][0][0]
        else:
            price = tr['bids'][0][0]
            
        return {"createdAt": time_string,
                "filledSize": 0,
                "future": market,
                "id": int(time.time()),
                "market": market,
                "price": price,
                "remainingSize": size,
                "side": side,
                "size": size,
                "status": "open",
                "type": type_ord,
              }
    
    def get_open_conditional_order(self,market):
        return self._get(f'/conditional_orders?market={market}')
    
    def get_conditional_order(self,conditional_order_id):
        return self._get(f'/conditional_orders/{conditional_order_id}/triggers')

In [9]:
############################################################################################
#------------------------------------  symbol class   -------------------------------------#
############################################################################################

In [76]:
class symbol():
    def __init__(self,sym,API):
        self.symbol = sym
        self.ticker = {'ask':'','bid':'','askv':'','bidv':''}
        self.his_price={}
        self.API = API
        
    def get_ticker(self):
        try:
            ticker = self.API.get_ticker(self.symbol)
            self.ticker['bid'] = float(ticker['bids'][0][0])
            self.ticker['bidv'] = float(ticker['bids'][0][1])
            self.ticker['ask'] = float(ticker['asks'][0][0])
            self.ticker['askv'] = float(ticker['asks'][0][1])
            return True
        except:
            return False
        
    def getHisPrice(self,tf,nbar):
        price = self.API.historicalPrice(self.symbol,tf,nbar,int(time.time())-(tf*nbar),int(time.time()))
        self.his_price = pd.DataFrame.from_dict(price)     
        
    def ma(self,type_bar):
        return self.his_price[type_bar].mean()

    def refBar(self,type_bar,bar):
        return self.his_price[type_bar][len(self.his_price)-bar]
    
    def atr(self):
        tr=[]
        for i in range(len(self.his_price)):
            tr.append(
                max(
                    (self.his_price['high'][i]-self.his_price['low'][i]),
                    abs(self.his_price['high'][i]-self.his_price['close'][i]),
                    (self.his_price['low'][i]-self.his_price['close'][i])
                )) 
        return np.mean(tr)

In [11]:
###################################################################################
#---------------------------------  main program  --------------------------------#
###################################################################################

In [12]:
class main():
    def __init__(self,sym_list,lot,margin):
        #self.API = bitkubAPI(API_HOST,API_KEY,API_SECRET)
        self.API = ftxAPI()
        self.order = {}
        self.margin = margin
        self.lot = lot
        self.atr = 1
        self.ticker = {}
        self.fee = 0.0007
        self.zone = 0
        self.timeframe = 5
        self.system = False
        
        
        #time
        self.tm = time.localtime() # get struct_time
        self.time_string = time.strftime("%Y-%m-%d, %H:%M:%S", self.tm)
        self.refSec = 0
        
        #HisPrice init
        self.sys = list([])
        for i in range(len(sym_list)):
            self.sys.append(i)
        self.calHisPrice()
            
    ########################### write data ###########################      
    def create_logfile (self):
        if(os.path.isfile('log.csv') == False):
            column_names = [  'zone',
                  'status',
                  'open_id',
                  'open_date',
                  'open_price',
                  'side',
                  'size',
                  'sl',
                  'tp',
                  'fee',
                  'close_id',
                  'close_date',
                  'close_price',
                  'comment',
                  'profit',
                  'zProfit']
            df = pd.DataFrame(columns = column_names)
            df.to_csv('log.csv',index=False)
            
    def write_log(self,dict_order): 
        with open('log.csv', 'a') as csv_object: 
            writer(csv_object).writerow(dict_order)  
            csv_object.close()       
            
    def load_order (self):
        if(os.path.isfile('data.json') == True):
            with open('data.json') as infile:
            self.order = json.load(infile)   
            
    def save_order(self): 
        with open('data.json') as outfile: 
            json.dump(self.order, outfile)  
        
    ########################### getdata ###########################   
    def time_check(self):
        #get_time
        self.tm = time.localtime() # get struct_time
        self.time_string = time.strftime("%Y-%m-%d, %H:%M:%S", self.tm)
        return True
    
    def calHisPrice(self)
        get_hisdata = list()
        for i in self.sys:
            get_hisdata = threading.Thread(target=self.sys[i].getHisPrice, args=[self.timeframe*60, 100])
            get_hisdata.start()
            get_hisdata.append(get_hisdata)
        for i in get_hisdata:
            i.join()

        for i in range(len(self.sys)):
            self.sys[i] = symbol(sym_list[i],self.API)
            self.sys[i].getHisPrice(self.timeframe*60,100)
            if i == 0: 
                self.ma = self.sys[i].ma('close')
                self.refBar = self.sys[i].refBar('high',1)
            elif i == 1: 
                self.ma = round(self.ma - self.sys[i].ma('close') , 5)
                self.ma = round(self.ma - self.sys[i].refBar('high',1) , 5)
            else:
                print('getHisPrice function error')
                
    def getHisPrice(self):
        if(self.refSec != self.tm and self.tm.tm_sec % self.timeframe  == 0):
            self.refSec = self.tm.tm_sec
            self.calHisPrice()
    
    ########################### open order ###########################             
    def place_orders_open(self,sym,side,size,price):
        res = self.API.place_orders(sym,side,size,price,'market')
        return{ 'zone': self.zone,
                'status':'open',
                'open_id' : res['id'],
                'open_date': res['createdAt'],
                'open_price': res['price'],
                'side':side,
                'size': res['size'],
                'sl': '',
                'tp': '',
                #'fee':self.fee*(res1['price']*res1['size']),
                'fee': 0,
                'comment':'',
                }

                        
    #-----long order-----              
    def long_open_conditon(self):
        #------check short_conditon
        long_conditon = all([  self.ticker['ask'] <= self.ma,
                               self.ticker['ask'] == self.zone,
                               self.ticker['ask'] not in self.order.keys()
                          ])
        
        if(self.time_check() and long_conditon):
            dict_order=list([0,1])
            dict_order[0] = self.place_orders_open(self.sys[0].symbol,'buy',self.lot,self.ticker['ask'])
            dict_order[1] = self.place_orders_open(self.sys[1].symbol,'sell',self.lot,self.ticker['bid'])

            self.order[self.zone][self.sys[0].symbol] = dict_order[0]
            self.order[self.zone][self.sys[1].symbol] = dict_order[1]
            self.save_order()
            
            #--test
            print(f'---------------------open long order {self.zone}----------------------')
            print(dict_order1)
            print(dict_order2)
            print('--------------------------------------------------------------------------------')
            
    #-----short order-----            
    def short_open_conditon(self):
        #------check short_conditon
        short_conditon = all([    self.ticker['ask'] >= self.ma,
                               self.ticker['ask'] == self.zone,
                               self.zone not in self.order.keys()
                          ])
        #--test
        if(self.time_check() and short_conditon):
        #if(True):
            dict_order=list([0,1])
            dict_order[0] = self.place_orders_open(self.sys[0].symbol,'sell',self.lot,self.ticker['bid'])
            dict_order[1]= self.place_orders_open(self.sys[1].symbol,'buy',self.lot,self.ticker['ask'])

            self.order[self.zone][self.sys[0].symbol] = dict_order[0]
            self.order[self.zone][self.sys[1].symbol] = dict_order[1]
            self.save_order()

            #--test
            print(f'---------------------open short order {self.zone}----------------------')
            print(dict_order1)
            print(dict_order2)
            print('--------------------------------------------------------------------------------')

            
    ########################### close_order ###########################       
    #-----place orders------
    def place_orders_close(self,sym,side,size,price,sym_no,zone,comment):
        res = self.API.place_orders(sym,side,size,price,'market')
        self.order[zone][sym_no]['status'] = 'close'
        self.order[zone][sym_no]['close_id'] = res["id"]
        self.order[zone][sym_no]['close_date'] = res["createdAt"]
        self.order[zone][sym_no]['close_price'] = res["price"]
        self.order[zone][sym_no]['profit'] = res["price"] - self.order[zone][sym_no]['open_price'] - (self.order[zone][sym_no]['fee']*2)
        self.order[zone][sym_no]['comment'] = f'close {comment}'
        
    #-----process close_order---------
    def process_closeOrder(self,zone):
        if(self.order[zone]['symbol_0']['side'] == 'buy'
          and self.ticker['bid'] >= zone + self.margin):
            #----thread set 
            thr_sym_0_place_orders = threading.Thread(target=self.place_orders_close, args=[self.symbol_0.symbol,'sell',self.lot,self.ticker['bid'],'symbol_0',zone,self.ticker['ask']])
            thr_sym_1_place_orders = threading.Thread(target=self.place_orders_close, args=[self.symbol_1.symbol,'buy',self.lot,self.ticker['ask'],'symbol_1',zone,self.ticker['ask']])
            #----thread start 
            thr_sym_0_place_orders.start()
            thr_sym_1_place_orders.start()
            #----thread join
            thr_sym_0_place_orders.join()
            thr_sym_1_place_orders.join()
            
            #cal zProfit
            self.order[zone]['symbol_0']['zProfit'] = self.order[zone]['symbol_0']['profit'] + self.order[zone]['symbol_1']['profit']
            self.order[zone]['symbol_1']['zProfit'] = self.order[zone]['symbol_0']['profit'] + self.order[zone]['symbol_1']['profit']
            #----thread set 
            thr_sym_0_create_log = threading.Thread(target=self.symbol_0.create_log, args=[self.order[zone]['symbol_0']])
            thr_sym_1_create_log = threading.Thread(target=self.symbol_1.create_log, args=[self.order[zone]['symbol_1']])
            #----thread start 
            thr_sym_0_create_log.start()
            thr_sym_1_create_log.start()
            #--test

            print(f'---------------------open long order {current_price}----------------------')
            print(self.order[zone]['symbol_0'])
            print(self.order[zone]['symbol_1'])
            print('----------------------------------------------------------------------------------')

            self.symbol_0.order.remove(self.order[zone]['symbol_0'])
            self.symbol_1.order.remove(self.order[zone]['symbol_1'])

            del self.order[zone]


        if(self.order[zone]['symbol_0']['side'] == 'sell' 
           and self.ticker['ask'] <= zone - self.margin):
            #----thread set 
            thr_sym_0_place_orders = threading.Thread(target=self.place_orders_fn, args=[self.symbol_0.symbol,'buy',self.lot,self.ticker['ask'],'symbol_0',zone,self.ticker['ask']])
            thr_sym_1_place_orders = threading.Thread(target=self.place_orders_fn, args=[self.symbol_1.symbol,'sell',self.lot,self.ticker['bid'],'symbol_1',zone,self.ticker['ask']])
            #----thread start 
            thr_sym_0_place_orders.start()
            thr_sym_1_place_orders.start()
            #----thread join
            thr_sym_0_place_orders.join()
            thr_sym_1_place_orders.join()
            
            #cal zProfit
            self.order[zone]['symbol_0']['zProfit'] = self.order[zone]['symbol_0']['profit'] + self.order[zone]['symbol_1']['profit']
            self.order[zone]['symbol_1']['zProfit'] = self.order[zone]['symbol_0']['profit'] + self.order[zone]['symbol_1']['profit']
            #----thread set 
            thr_sym_0_create_log = threading.Thread(target=self.symbol_0.create_log, args=[self.order[zone]['symbol_0']])
            thr_sym_1_create_log = threading.Thread(target=self.symbol_1.create_log, args=[self.order[zone]['symbol_1']])
            #----thread start 
            thr_sym_0_create_log.start()
            thr_sym_1_create_log.start()
            #--test

            print(f'---------------------open short order {current_price}----------------------')
            print(self.order[zone]['symbol_0'])
            print(self.order[zone]['symbol_1'])
            print('----------------------------------------------------------------------------------')

            self.symbol_0.order.remove(self.order[zone]['symbol_0'])
            self.symbol_1.order.remove(self.order[zone]['symbol_1'])

            del self.order[zone]

    #-----operation------
    def close_order(self):
        processes = list()
        for zone in self.order.keys():
            process = threading.Thread(target=self.process_closeOrder, args=[zone])
            process.start()
            processes.append(process)
        for process in processes:
            process.join()
            
            
    ######################## initialize ########################       
    def initialize(self):              
        self.create_logfile()
    ########################### start ###########################     
    def start(self):
        try:
            #get_ticker
            self.system = all([self.symbol_0.get_ticker(),self.symbol_1.get_ticker()])
            self.ticker['ask']  = round(self.symbol_0.ticker['ask'] - self.symbol_1.ticker['bid'], 5)
            self.ticker['bid']  = round(self.symbol_0.ticker['bid'] - self.symbol_1.ticker['ask'], 5)
            
            #----thread set 
            thr_getHisPrice = threading.Thread(target=self.getHisPrice)
            #----thread start 
            thr_getHisPrice.start()
            self.system = True
        except:
            self.system = False


        if(self.system):
            #cal zone
            self.zone = round((self.ticker['ask'] // self.margin) * self.margin,5)
            
            #----thread set 
            #------open long condition ------
            thr_open_long = threading.Thread(target=self.long_open_conditon)
            #------check short_conditon
            thr_open_short = threading.Thread(target=self.short_open_conditon)
               
            #----thread start 
            thr_open_long.start()
            thr_open_short.start()
            #----thread join 
            thr_open_long.join()
            thr_open_short.join()
            
            #------check close_conditon
            self.close_order()
            
            ask  = self.ticker['ask']
            print(f'{self.symbol_0.symbol}/{self.symbol_1.symbol} zone:{self.zone} ask:{ask} ma:{self.ma} {self.time_string}    ')#,end='\r')
        else:
            print(f'connection failed {self.time_string}                                                                        ')#,end='\r')


In [13]:
program = main(['XRP-PERP','XLM-PERP'],1,0.0005)
program.getOrder()

In [14]:
i=0
while(i!=50):
    clear_output(wait=True)
    print(f'------------------{i}---------------------')
    program.start()
    i += 1
    print('-------------------------------------------')
    for j in program.order:
        print(f'{j} {program.order[j]}')
        print('')
    time.sleep(0)
    

Exception in thread Thread-273:
Traceback (most recent call last):
  File "C:\Users\CAFQuant\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\CAFQuant\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-d6acb412e44a>", line 49, in create_log
  File "C:\Users\CAFQuant\anaconda3\lib\site-packages\gspread\models.py", line 314, in worksheet
    sheet_data = self.fetch_sheet_metadata()
  File "C:\Users\CAFQuant\anaconda3\lib\site-packages\gspread\models.py", line 266, in fetch_sheet_metadata
    r = self.client.request('get', url, params=params)
  File "C:\Users\CAFQuant\anaconda3\lib\site-packages\gspread\client.py", line 76, in request
    raise APIError(response)
gspread.exceptions.APIError: {'code': 429, 'message': "Quota exceeded for quota metric 'Read requests' and limit 'Read requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:460080090

------------------49---------------------


Exception in thread Thread-274:
Traceback (most recent call last):
  File "C:\Users\CAFQuant\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\CAFQuant\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-d6acb412e44a>", line 49, in create_log
  File "C:\Users\CAFQuant\anaconda3\lib\site-packages\gspread\models.py", line 314, in worksheet
    sheet_data = self.fetch_sheet_metadata()
  File "C:\Users\CAFQuant\anaconda3\lib\site-packages\gspread\models.py", line 266, in fetch_sheet_metadata
    r = self.client.request('get', url, params=params)
  File "C:\Users\CAFQuant\anaconda3\lib\site-packages\gspread\client.py", line 76, in request
    raise APIError(response)
gspread.exceptions.APIError: {'code': 429, 'message': "Quota exceeded for quota metric 'Read requests' and limit 'Read requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:460080090

XRP-PERP/XLM-PERP zone:0.1315 ask:0.13187 ma:0.11533 2021-03-22, 17:21:24    
-------------------------------------------


In [ ]:
program.symbol

In [ ]:
program.symbol_0.order

In [ ]:
que = Queue

In [16]:
jsonTest = {'a':1,'b'}

In [17]:
with open('data.txt', 'w') as outfile:
    json.dump(jsonTest, outfile)

In [18]:
with open('data.txt') as json_file:
    data = json.load(json_file)

In [19]:
class abc()
    

{'a': 1, 'b': 2}

In [21]:
# Path  
path = 'data.txt'
    
# Check whether the   
# specified path is   
# an existing file  


In [28]:
pd.DataFrame({'' : []})

,123


In [24]:
'zone','status','open_id','open_date','open_price','side','size','sl','tp','fee','close_id','close_date','close_price','comment','profit','zProfit'

,col1,col2
0,1,3
1,2,4


In [79]:
sym_list = ['XRP-PERP','XLM-PERP']
timeframe = 5

In [83]:
sys = list([])
for i in range(len(sym_list)):
    sys.append(i)
for i in range(len(sys)):
    sys[i] = symbol(sym_list[i],ftxAPI())
    sys[i].getHisPrice(timeframe*60,100)
    if i == 0: 
        ma = sys[i].ma('close')
        refBar = sys[i].refBar('high',1)
    elif i == 1: 
        ma = round(ma - sys[i].ma('close') , 5)
        ma = round(ma - sys[i].refBar('high',1) , 5)
            

In [72]:
sys 

TypeError: 'str' object is not callable

In [78]:
z= symbol('XRP-PERP',ftxAPI())